# Image Classification using Quantum Kernel and SVC

This program demonstrates the implementation of Quantum kernel-based training of a Support Vector classifier.The kernel, which is a small building block of the algorithm, is computed by a quantum device and is executed repetitively. For quantum machine learning applications with many parameters, kernel-based training can be a great alternative to the variational approach to quantum machine learning.

Normal and cataract eye images are converted to grayscale, resized to 28X28 and their dimensionality further reduced through kernel principal component analysis, which captures the non-linear features and pares down the dimensions to say, 4 to 6. This  number decides the number of qubuts in the quantum kernel. The quantum kernel is given by the mutual overlap of two data-encoding quantum states. In this example, angle-encoding is used to prepare the two states and the quantum node is used as a quantum kernel evaluator. The Support Vector Machine is trained using the kernel matrix with the labelled training data.   

Ref: Pennylane demo 'Kernel-based training of quantum models with scikit-learn'

In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

In [2]:
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [3]:
import os, glob, cv2
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_custom_objects

In [4]:
SEED = 42
IMG_HEIGHT =28
IMG_WIDTH = 28

# eye_diseases_classification dataset
IMG_ROOT = 'D:\Womanium2023\GlobalQuantumProject\Datasets\eye_diseases_classification\Proc\\'
IMG_DIR = [IMG_ROOT+'normal',
           IMG_ROOT+'cataract']

In [5]:
IMG_DIR

['D:\\Womanium2023\\GlobalQuantumProject\\Datasets\\eye_diseases_classification\\Proc\\normal',
 'D:\\Womanium2023\\GlobalQuantumProject\\Datasets\\eye_diseases_classification\\Proc\\cataract']

In [6]:
def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

In [7]:
df = pd.DataFrame(0,
                  columns=['paths',
                           'cataract'],
                  index=range(2500))

filepaths = glob.glob(IMG_ROOT + '*/*')

for i, filepath in enumerate(filepaths):
    filepath = os.path.split(filepath)
    df.iloc[i, 0] = filepath[0] + '/' + filepath[1]

    if filepath[0] == IMG_DIR[0]:    # normal
            df.iloc[i, 1] = 0
    elif filepath[0] == IMG_DIR[1]:  # cataract
            df.iloc[i, 1] = -1

In [8]:
print('Number of normal and cataract images')
print(df['cataract'].value_counts())

Number of normal and cataract images
cataract
 0    1562
-1     938
Name: count, dtype: int64


In [9]:
df = df[df.paths !=0]

In [26]:
train_df, test_df = train_test_split(df,
                                     test_size=0.2,
                                     random_state=SEED,
                                     stratify=df['cataract'])

train_df, val_df = train_test_split(train_df,
                                    test_size=0.15,
                                    random_state=SEED,
                                    stratify=train_df['cataract'])

In [27]:
from tqdm import tqdm

def create_datasets(df):
    imgs = []

    for path in tqdm(df['paths']):
        #print(path)
        img = cv2.imread(path,0)
        img = cv2.resize(img,(IMG_HEIGHT,IMG_WIDTH))
        img=img.flatten() #convert 2d to 1d array
        imgs.append(img)

    imgs = np.array(imgs, dtype='float32')

    labels = df.cataract
    return imgs, labels


train_imgs, train_labels = create_datasets(train_df)
val_imgs, val_labels = create_datasets(val_df)
test_imgs, test_labels = create_datasets(test_df)

train_imgs = train_imgs / 255.0
val_imgs = val_imgs / 255.0
test_imgs = test_imgs / 255.0



100%|██████████████████████████████████████████████████████████████████████████████| 403/403 [00:00<00:00, 7427.66it/s]


In [29]:
np.shape(test_imgs)

(403, 784)

In [13]:
n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 1000   # Size of the train dataset
n_test = 50     # Size of the test dataset

SAVE_PATH = "eyeQCNN1/" # Data saving folder
PREPROCESS = False           # If False, skip quantum processing and load data from SAVE_PATH
np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)       # Seed for TensorFlow random number generator

In [14]:
# Reduce dataset size
train_images = train_imgs[:n_train]
train_labels = train_labels[:n_train]
test_images = test_imgs[:n_test]
test_labels = test_labels[:n_test]

In [15]:
from sklearn.decomposition import KernelPCA
#from sklearn import preprocessing

kernel_pca = KernelPCA(
    n_components=4, kernel="rbf", gamma=None, fit_inverse_transform=True, alpha=0.1
)

# rescale the data so it has unit standard deviation and zero mean.
#scaler = preprocessing.StandardScaler().fit(data)
#data = scaler.transform(data)

X_train= kernel_pca.fit(train_images).transform(train_images)
testfit = kernel_pca.transform(test_images)

In [16]:
n_qubits = len(X_train[0])
n_qubits

4

In [17]:
dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [18]:
kernel(X_train[0], X_train[0])#sanity check

array(1.)

In [19]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [20]:
svm = SVC(kernel=kernel_matrix).fit(X_train, train_labels)

In [21]:
predictions = svm.predict(testfit)
accuracy_score(predictions, test_labels)

0.72